# **Tutorial - Week 7 - Blockchain**

**Blockchain**: A digital database or ledger that is distributed among the nodes of peer-to-peer network

<img src="https://qph.cf2.quoracdn.net/main-qimg-d9d88c0959769425615aaf5cc30835b4" width=600>

## 1. Hash function

In this week’s lecture, you learned about the hash function, cryptographic hash functions, and their applications.

**What is Hash Function ?**

A type of cryptography that takes in a random value as input and gives back a fixed length value as output.

<img src="https://miro.medium.com/v2/resize:fit:2000/1*61uolxM5HsroqBFaQxsGxQ.png" width=500>

- **Deterministic**: The same input will give the same output
- **One-way function**: A hash can be easily computed for an input, but it is infeasible to reverse it.
- **Collision-free**: No two input hashes should map to the same output hash.

With the help of Python libraries like `hashlib`, it is simple to calculate the hash value with standard hash functions such as the **SHA-256** hashing algorithm. Below is a sample code to calculate an **SHA-256** hash of message 12345 using hashlib and print the hash value encoded.

In [ ]:
import hashlib

# encode message to bytes using UTF-8 encoding
message = "12345".encode("utf-8")
# hash with SHA-256
rawHash = hashlib.sha256(message)
# return as a string containing only hexadecimal digits
hexHash = rawHash.hexdigest()

print("Hash value:", hexHash)
print("Bit Length:", len(hexHash) * 4)

Hash value: 5994471abb01112afcc18159f6cc74b4f511b99806da59b3caf5a9c173cacfc5
Bit Length: 256


## 2. Transaction

A blockchain is a linked list of blocks, each containing transactions. A transaction is defined as the container to store a single message (sender + content).


<img src="https://www.researchgate.net/publication/342527999/figure/tbl1/AS:907748308959239@1593435470930/Transaction-Format-of-the-Blockchain-System.png" width=500>


In our implementation, the transaction is received in JSON format as an object with the following keys: sender and message, with string values. You should perform some checks before considering a message as a valid transaction. Here are the requirements for a message to be a valid transaction:

- The message sender must be present and should **match a 64-character hex string** (regex: [a-fA-F0-9]{64})
  + []: set of characters
  + ^ : starts with
  + a-fA-F0-9: any character alphabetically between a and f, lower case OR upper case, or any digit between 0 and 9
  + {}: exactly the specified number of occurrences
- The message content must be present and **cannot have more than 70 alphanumeric characters**

If a transaction violates these rules, it should be considered as an invalid transaction.

You can see an example of a valid transaction below

In [ ]:
transaction = {
    'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b',
    'message': 'hello'
}

#### Task

Use the skeleton code in the supplied `blockchain.py` file available on Canvas to implement `make_transaction` and `validate_transaction` functions.

- `make_transaction` takes sender and message arguments and returns a JSON string.
- `validate_transaction` takes JSON string argument, checks if the supplied string meets the requirements specified above, and returns a transaction dictionary, or an appropriate error value.

In [ ]:
from enum import Enum
import hashlib
import json
import re

# []: set of characters
# ^ : starts with
# a-fA-F0-9: any character alphabetically between a and f, lower case OR upper case, or any digit between 0 and 9
# {}: exactly the specified number of occurrences
sender_valid = re.compile('^[a-fA-F0-9]{64}$')

# Define Transaction Validation Error
TransactionValidationError = Enum('TransactionValidationError', ['INVALID_JSON', 'INVALID_SENDER', 'INVALID_MESSAGE'])

def make_transaction(sender, message) -> str:
	# TODO creation of JSON string from the dictionary containing the sender and the message
	pass

def validate_transaction(transaction: str) -> dict | TransactionValidationError:
	# TODO implement JSON string loading, and sender and message validation
	pass


### Solution

In [ ]:
from enum import Enum
import hashlib
import json
import re

# []: set of characters
# ^ : starts with
# a-fA-F0-9: any character alphabetically between a and f, lower case OR upper case, or any digit between 0 and 9
# {}: exactly the specified number of occurrences
sender_valid = re.compile('^[a-fA-F0-9]{64}$')

TransactionValidationError = Enum('TransactionValidationError', ['INVALID_JSON', 'INVALID_SENDER', 'INVALID_MESSAGE'])

def make_transaction(sender, message) -> str:
	return json.dumps({'sender': sender, 'message': message})

def validate_transaction(transaction: str) -> dict | TransactionValidationError:
	try:
		tx = json.loads(transaction)
	except json.JSONDecodeError:
		return TransactionValidationError.INVALID_JSON

	if not(tx.get('sender') and isinstance(tx['sender'], str) and sender_valid.search(tx['sender'])):
		return TransactionValidationError.INVALID_SENDER

	if not(tx.get('message') and isinstance(tx['message'], str) and len(tx['message']) <= 70 and tx['message'].isalnum()):
		return TransactionValidationError.INVALID_MESSAGE

	return tx

### Test Transaction

In [ ]:
# Create a transaction as dictionary
sender = 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b'
message = 'hello'

transaction_1 = make_transaction(sender, message)

print(transaction_1)

print("Transaction valid :", validate_transaction(transaction_1))

{"sender": "a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b", "message": "hello"}
Transaction valid : {'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b', 'message': 'hello'}


In [ ]:
sender = '@57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b',
message = 'hello'

transaction_2 = make_transaction(sender, message)

print(transaction_1)

print("Transaction valid :", validate_transaction(transaction_2))

{"sender": "a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b", "message": "hello"}
Transaction valid : TransactionValidationError.INVALID_SENDER


## 3. Block

In the lectures, you have seen the structure of blockchain.

<img src="https://community.sap.com/legacyfs/online/storage/blog_attachments/2019/01/B_1.jpg" width=600>

- An **ordered, back-linked list** of blocks
- The first block in any blockchain is called **the genesis block**

Each block contains the following fields:

- a list of transactions
- hash of the previous block
- block index in the blockchain



The important thing here is to calculate the hash of each block precisely. For example, we can take the three block fields "index", "transactions", and "previous_hash", serialize them in JSON format with keys sorted lexicographically using the json.dumps method and the sort_keys parameter, and use the resulting string as an input to the hash function. This will guarantee that the string for each block will always be the same.

**Data**: Previous_Hash, Transaction Pool, Index

**Result**: Current_Hash

Block_String = serialize(Previous_Hash, Pool, Index);

Current_Hash = hash(Block_String);

Within the scope of this tutorial, we will use SHA-256 as the hash function. You will be required to implement hash calculation function for blockchain in the next section.

You can see an example of a block below.

In [ ]:
block = {
  'index': 2,
  'transactions': [{'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b',
                    'message': 'hello'
                    }],
  'previous_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1',
  'current_hash': 'a79cc460599211dfadb73afc81ce2129cb55df9c43d5f27e522041fab9c8ed35'
  }

You can use the hash values in the example to test your implementation

## 4. Blockchain

A blockchain implementation contains the blockchain itself (i.e., the committed blocks of transactions) and a transaction pool to store uncommitted transactions. More precisely, it contains:

- a list of valid uncommitted transactions stored in a transaction pool;
- the latest block of the blockchain;
- the length of the blockchain.

<img src="https://blockchain.uvt.ro/wp-content/uploads/2022/01/transactions.png" width=600>

**Example - Bitcoin Mining**

1. A transaction is sent and put into a temporary pool where from there is getting into a block.
2. Miners get those transactions from the pool and mine the next hash for the block.
  - *Mining Difficulty*: how difficult it is to find a hash below a given target.
  - For the hash to be valid, it must be equal to or smaller than the target value. The larger the target value is, the harder it is for miners to find a valid hash.
  - Eg.a target hash: `0x0404cb * 2**(8*(0x1b - 3)) = 0x00000000000404CB000000000000000000000000000000000000000000000000`
3. Once the next hash has been found the block is created with the transactions from the pool.



The blockchain has an `add_transaction(self, transaction)` method to add a transaction to the transaction pool. First, you need to check the attributes of the `transaction` following the format described in Section 2. Then, if a transaction is valid, it is added to the local transaction pool. If the transaction is added to the pool, the method should return `True`, and `False` otherwise. `new_block(self, previous_hash)` method commits all uncommitted transactions in the pool to a new block, and appends that block to the blockchain. When a new block is generated, the length of the blockchain should increase by 1. Genesis block has index 1 and `previous_hash` 0.

### Task

Use the skeleton code in the supplied blockchain.py file to implement a Blockchain class.

You need to write code for some methods in the Blockchain class including:

- `new_block(self, previous_hash)`: append a new block consisting of the following properties:
  + Index: the index of the block in blockchain
  + Transaction Pool: an array containing transactions
  + Previous Hash: the hash value of the previous block
  + Current Hash: the hash value of the current block
- `calculate_hash(self, block)`: calculate hash value using SHA-256 for a block object using the method described in Section 3.

In [ ]:
class Blockchain():
	def  __init__(self):
		self.blockchain = []
		self.pool = []
		self.new_block('0' * 64)

	def new_block(self, previous_hash=None):
		# TODO create a block dict with the correct fields, clear the pool, and append the block to the blockchain
    # 1. Define block
    # 2. Calculate current hash
    # 3. Append block to the blockchain
		pass

	def last_block(self):
		return self.blockchain[-1]

	def calculate_hash(self, block: dict) -> str:
		# TODO implement the hash calculation by using the correct fields of the block and JSON serialization, and return hex string of the hash
    # 1. Convert block to string object
    # 2. Using hashlib.sha256 to compute hash
    # 3. Convert to raw hash to hexadecimal digits
		pass

	def add_transaction(self, transaction: str) -> bool:
		if isinstance((tx := validate_transaction(transaction)), dict):
			self.pool.append(tx)
			return True
		return False

### Solution

In [ ]:
class Blockchain():
	def  __init__(self):
		self.blockchain = []
		self.pool = []
		self.new_block('0' * 64)

	def new_block(self, previous_hash=None):
   # TODO create a block dict with the correct fields, clear the pool, and append the block to the blockchain
		block = {
			'index': len(self.blockchain) + 1,
			'transactions': self.pool.copy(),
			'previous_hash': previous_hash or self.blockchain[-1]['current_hash'],
		}
		block['current_hash'] = self.calculate_hash(block)
		self.pool = []
		self.blockchain.append(block)

	def last_block(self):
		return self.blockchain[-1]

	def calculate_hash(self, block: dict) -> str:
    # TODO implement the hash calculation by using the correct fields of the block and JSON serialization, and return hex string of the hash
		block_object: str = json.dumps({k: block.get(k) for k in ['index', 'transactions', 'previous_hash']}, sort_keys=True)
		block_string = block_object.encode()
		raw_hash = hashlib.sha256(block_string)
		hex_hash = raw_hash.hexdigest()
		return hex_hash

	def add_transaction(self, transaction: str) -> bool:
		if isinstance((tx := validate_transaction(transaction)), dict):
			self.pool.append(tx)
			return True
		return False


#### Test Blockchain

In [ ]:
# Create a block as dictionary
B = Blockchain()
print(B.blockchain)

[{'index': 1, 'transactions': [], 'previous_hash': '0000000000000000000000000000000000000000000000000000000000000000', 'current_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1'}]


In [ ]:
sender = 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b'
message = 'hello'

transactions = make_transaction(sender, message)

B.add_transaction(transactions)
print(B.pool)

[{'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b', 'message': 'hello'}]


In [ ]:
print("Add new block")
B.new_block()
print(B.blockchain)

Add new block
[{'index': 1, 'transactions': [], 'previous_hash': '0000000000000000000000000000000000000000000000000000000000000000', 'current_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1'}, {'index': 2, 'transactions': [{'sender': 'a57819938feb51bb3f923496c9dacde3e9f667b214a0fb1653b6bfc0f185363b', 'message': 'hello'}], 'previous_hash': '03525042c7132a2ec3db14b7aa1db816e61f1311199ae2a31f3ad1c4312047d1', 'current_hash': 'a79cc460599211dfadb73afc81ce2129cb55df9c43d5f27e522041fab9c8ed35'}]
